In [1]:

import numpy as np
import pandas as pd
import time
import os 
import random
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import mutual_info_regression,mutual_info_classif
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.model_selection import cross_validate, KFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score as ACC 
from sklearn.metrics import log_loss as logloss 
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.linear_model import LogisticRegression as LogiR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC
# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *
from tqdm import tqdm
import gc
#日常使用库与算法
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
#导入优化算法
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss
#融合模型
from sklearn.ensemble import StackingClassifier

In [2]:
data = pd.read_csv("./6_1data_exp_shifted.csv",index_col=0)

data_test=pd.read_csv('./6_1data_exp_test_shifted.csv',index_col=0)

features = data.drop(columns='group').copy()
labels = data['group'].copy()

features_test = data_test.drop(columns='group').copy()
labels_test = data_test['group'].copy()

shift_gene=['ENSG00000164600.4', 'ENSG00000095970.12', 'ENSG00000171860.4',
       'ENSG00000135914.5', 'ENSG00000075275.12','ENSG00000131095.7']

features_keep_test=features_test.loc[:,shift_gene]

features_keep=features.loc[:,shift_gene]

scaler = StandardScaler()

scall=scaler.fit(features_keep)

ff_keep=scall.transform(features_keep)

ff_keep_test=scall.transform(features_keep_test)

ff_keep=pd.DataFrame(ff_keep)

ff_keep.columns=shift_gene

ff_keep_test=pd.DataFrame(ff_keep_test)

ff_keep_test.columns=shift_gene

In [3]:
Poly_train, Poly_test, colNames_train_new, colNames_test_new = Polynomial_Features(shift_gene, 
                                                                                   3, 
                                                                                   ff_keep, 
                                                                                   ff_keep_test, 
                                                                                   multi=False)

In [4]:
MI = mutual_info_classif(Poly_train, labels, random_state=1412)


MI_threshold = MI.mean() * 0.1

MI_Ploy_cols = []

for MIvalue, colname in zip(MI, colNames_train_new):
    if MIvalue > MI_threshold:
        MI_Ploy_cols.append(colname)

print(len(MI_Ploy_cols))        

93


In [5]:
Poly_train=Poly_train.loc[:,MI_Ploy_cols]
Poly_test=Poly_test.loc[:,MI_Ploy_cols]

In [6]:
Poly_train=pd.concat([Poly_train,ff_keep],axis=1)
Poly_test=pd.concat([Poly_test,ff_keep_test],axis=1)

In [7]:
#日常使用库与算法
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
#导入优化算法
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

In [8]:
def hyperopt_objective(params):
    reg = XGBClassifier(n_estimators = int(params["n_estimators"]) #
              ,learning_rate = params["lr"]   #
              ,booster = params['booster'] #
              ,gamma = params["gamma"]
              ,max_depth = int(params["max_depth"])  #
              ,min_child_weight = params["min_child_weight"]
              ,subsample = params["subsample"]
              ,colsample_bytree = params["colsample_bytree"]
              ,colsample_bynode = params["colsample_bynode"]
              ,reg_lambda = params["reg_lambda"]
              ,random_state=420
              )
    validation_loss = cross_validate(reg,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=5
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')
    return -np.mean(abs(validation_loss["test_score"]))

In [9]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",80,150,1)
                  ,"lr": hp.quniform("learning_rate",0,0.4,0.01)
                  ,"booster": hp.choice("booster",['gbtree'])
                  ,"gamma":hp.quniform("gamma",3,7,0.01)
                  ,"max_depth": hp.quniform("max_depth",1,5,1)
                  ,"min_child_weight": hp.quniform("min_child_weight",2,5,0.1)
                  ,"subsample":hp.quniform("subsample",0.5,0.9,0.01)
                  ,"colsample_bytree":hp.quniform("colsample_bytree",0,1,0.01)
                  ,"colsample_bynode":hp.quniform("colsample_bynode",0,1,0.01)
                  ,"reg_lambda":hp.quniform("reg_lambda",1.5,4,0.01)
                 }

In [10]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(100)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [11]:
params_best, trials = param_hyperopt(1000)

 25%|██████████▊                                 | 246/1000 [01:06<03:23,  3.70trial/s, best loss: -0.7558552274861385]

 
 best params:  {'booster': 0, 'colsample_bynode': 0.07, 'colsample_bytree': 0.96, 'gamma': 5.98, 'learning_rate': 0.17, 'max_depth': 1.0, 'min_child_weight': 3.8000000000000003, 'n_estimators': 108.0, 'reg_lambda': 2.69, 'subsample': 0.67} 



In [12]:
params_best, trials = param_hyperopt(1000)

 27%|███████████▋                                | 267/1000 [01:16<03:29,  3.50trial/s, best loss: -0.7548424198360049]

 
 best params:  {'booster': 0, 'colsample_bynode': 0.09, 'colsample_bytree': 0.72, 'gamma': 3.27, 'learning_rate': 0.14, 'max_depth': 1.0, 'min_child_weight': 2.6, 'n_estimators': 141.0, 'reg_lambda': 2.25, 'subsample': 0.8200000000000001} 



In [45]:
params_best, trials = param_hyperopt(1000)

 11%|████▋                                       | 106/1000 [00:28<03:59,  3.74trial/s, best loss: -0.7571260823728247]

 
 best params:  {'booster': 0, 'colsample_bynode': 0.88, 'colsample_bytree': 0.12, 'gamma': 3.56, 'learning_rate': 0.24, 'max_depth': 1.0, 'min_child_weight': 2.1, 'n_estimators': 91.0, 'reg_lambda': 3.99, 'subsample': 0.59} 



In [30]:
def hyperop_t(params):
    reg = XGBClassifier(n_estimators = int(params["n_estimators"]) #
              ,learning_rate = params["learning_rate"]   #
              ,booster =['gbtree'][params['booster']] #
              ,gamma = params["gamma"]
              ,max_depth = int(params["max_depth"])  #
              ,min_child_weight = params["min_child_weight"]
              ,subsample = params["subsample"]
              ,colsample_bytree = params["colsample_bytree"]
              ,colsample_bynode = params["colsample_bynode"]
              ,reg_lambda = params["reg_lambda"]
              ,random_state=1412
              )
    validation_loss = cross_validate(reg,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=5
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')
    return -np.mean(abs(validation_loss["test_score"]))

In [31]:
hyperop_t(params_best)

-0.7539069821715587

In [13]:
# 模型尝试 basic_model
clf1=XGBClassifier(n_estimators =91,learning_rate=0.24,booster='gbtree',gamma = 3.56,max_depth = 1,min_child_weight =2.1,subsample =0.59,colsample_bytree =0.12,colsample_bynode =0.88,reg_lambda =3.99,random_state=1412)
clf2=XGBClassifier(n_estimators =108,learning_rate=0.17,booster='gbtree',gamma = 5.98,max_depth = 1,min_child_weight =3.8,subsample =0.67,colsample_bytree =0.96,colsample_bynode =0.07,reg_lambda =2.69,random_state=420)

In [53]:
def hyperopt_objective(params):
    reg = GBC(n_estimators = int(params["n_estimators"])
              ,learning_rate = params["lr"]
              ,criterion = params["criterion"]
              ,loss = params["loss"]
              ,max_depth = int(params["max_depth"])
              ,max_features = params["max_features"]
              ,subsample = params["subsample"]
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=420
              ,verbose=False)
    validation_loss = cross_validate(reg,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=5
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')
    return -np.mean(abs(validation_loss["test_score"]))

In [54]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",30,100,1)
                  ,"lr": hp.quniform("learning_rate",0.01,0.3,0.01)
                  ,"criterion": hp.choice("criterion",['friedman_mse'])
                  ,"loss":hp.choice("loss",["deviance","exponential"])
                  ,"max_depth": hp.quniform("max_depth",1,7,1)
                  ,"subsample": hp.quniform("subsample",0.3,0.9,0.01)
                  ,"max_features": hp.choice("max_features",["sqrt",15,20,23,25,27,30,32,35])
                  ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1.5,0.01)
                 }

In [55]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(200)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [56]:
params_best, trials = param_hyperopt(1000)

 60%|██████████████████████████▎                 | 598/1000 [03:04<02:04,  3.24trial/s, best loss: -0.7603407883684428]

 
 best params:  {'criterion': 0, 'learning_rate': 0.16, 'loss': 1, 'max_depth': 1.0, 'max_features': 2, 'min_impurity_decrease': 0.88, 'n_estimators': 83.0, 'subsample': 0.67} 



In [47]:
params_best, trials = param_hyperopt(1000)

 81%|███████████████████████████████████▌        | 807/1000 [04:18<01:01,  3.13trial/s, best loss: -0.7595505685452358]

 
 best params:  {'criterion': 0, 'learning_rate': 0.15, 'loss': 1, 'max_depth': 1.0, 'max_features': 6, 'min_impurity_decrease': 0.98, 'n_estimators': 81.0, 'subsample': 0.72} 



In [57]:
clf3= GBC(n_estimators = 81
              ,learning_rate = 0.15
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 30
              ,subsample = 0.72
              ,min_impurity_decrease = 0.98
              ,random_state=1412
              ,verbose=False)

In [58]:
clf4= GBC(n_estimators = 83
              ,learning_rate = 0.16
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 20
              ,subsample = 0.67
              ,min_impurity_decrease = 0.88
              ,random_state=420
              ,verbose=False)

In [8]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [78]:
def hyperopt_objective(params):
    reg = RFC(n_estimators = int(params["n_estimators"])
              ,criterion = params["criterion"]
              ,max_depth = int(params["max_depth"])
              ,max_features = params["max_features"]
              ,max_samples = params["max_samples"]
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False)
    validation_loss = cross_validate(reg,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=5
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')
    return -np.mean(abs(validation_loss["test_score"]))

In [79]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",30,90,1)
                  ,"criterion": hp.choice("criterion",["gini","entropy"])
                  ,"max_depth": hp.quniform("max_depth",1,7,1)
                  ,"max_samples": hp.quniform("max_samples",0.3,0.9,0.01)
                  ,"max_features": hp.choice("max_features",["sqrt",15,18,20,23,25,27,30,32,35,40])
                  ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,1.5,0.01)
                 }

In [80]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(200)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [74]:
params_best, trials = param_hyperopt(1000)

 61%|██████████████████████████▊                 | 609/1000 [03:05<01:59,  3.28trial/s, best loss: -0.7402826479548094]

 
 best params:  {'criterion': 1, 'max_depth': 6.0, 'max_features': 8, 'max_samples': 0.63, 'min_impurity_decrease': 0.0, 'n_estimators': 74.0} 



In [81]:
params_best, trials = param_hyperopt(1000)

 29%|█████████████                                | 291/1000 [01:19<03:14,  3.64trial/s, best loss: -0.742746483926085]

 
 best params:  {'criterion': 1, 'max_depth': 5.0, 'max_features': 4, 'max_samples': 0.64, 'min_impurity_decrease': 0.0, 'n_estimators': 51.0} 



In [82]:
clf5=RFC(criterion='entropy',max_depth=5,max_features=23,max_samples=0.64,min_impurity_decrease=0,n_estimators=51,random_state=1412)

In [10]:
from sklearn.svm import SVC

In [13]:
def hyperopt_objective(params):
    reg = SVC(C = params["C"]
              ,kernel = params["kernel"]
              #,gamma=params['gamma']

              ,random_state=1412
              ,verbose=False)
    validation_loss = cross_validate(reg,Poly_train,labels
                                     ,scoring="roc_auc"
                                     ,cv=5
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise')
    return -np.mean(abs(validation_loss["test_score"]))

In [14]:
param_grid_simple = {'C': hp.quniform("C",0.01,5,0.01)
                  ,"kernel": hp.choice("kernel",["rbf",'poly','sigmoid','linear'])
                 # ,"gamma" :hp.quniform("gamma",0.01,5,0.01)

                 }

In [15]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(200)
    
    #定义代理模型
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

In [16]:
params_best, trials = param_hyperopt(1000)

 40%|█████████████████▍                          | 397/1000 [05:36<08:31,  1.18trial/s, best loss: -0.7265277720109136]

 
 best params:  {'C': 0.13, 'kernel': 3} 



In [9]:
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.linear_model import LogisticRegression as LogiR

In [42]:
clf6=SVC(C=0.13,kernel='linear')

In [43]:
clf7 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)

In [38]:
clf8 = KNNC(n_neighbors=20,n_jobs=8)

In [15]:
 # all basic model
clf1=XGBClassifier(n_estimators =91,learning_rate=0.24,booster='gbtree',gamma = 3.56,max_depth = 1,min_child_weight =2.1,subsample =0.59,colsample_bytree =0.12,colsample_bynode =0.88,reg_lambda =3.99,random_state=1412)
clf2=XGBClassifier(n_estimators =108,learning_rate=0.17,booster='gbtree',gamma = 5.98,max_depth = 1,min_child_weight =3.8,subsample =0.67,colsample_bytree =0.96,colsample_bynode =0.07,reg_lambda =2.69,random_state=420)
clf3= GBC(n_estimators = 81
              ,learning_rate = 0.15
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 30
              ,subsample = 0.72
              ,min_impurity_decrease = 0.98
              ,random_state=1412
              ,verbose=False)
clf4= GBC(n_estimators = 83
              ,learning_rate = 0.16
              ,criterion = 'friedman_mse'
              ,loss = "exponential"
              ,max_depth = 1
              ,max_features = 20
              ,subsample = 0.67
              ,min_impurity_decrease = 0.88
              ,random_state=420
              ,verbose=False)
clf5=RFC(criterion='entropy',max_depth=5,max_features=23,max_samples=0.64,min_impurity_decrease=0,n_estimators=51,random_state=1412)
clf6=SVC(C=0.13,kernel='linear')
clf7 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)
clf8 = KNNC(n_neighbors=20,n_jobs=8)
